In [1]:
import hmmlearn
import pybind11
import sys

print("hmmlearn version:", hmmlearn.__version__)
print("pybind11 version:", pybind11.__version__)
print("Python version:", sys.version)

hmmlearn version: 0.3.2
pybind11 version: 2.12.0
Python version: 3.9.15 (main, Nov 24 2022, 14:39:17) [MSC v.1916 64 bit (AMD64)]


In [2]:
import numpy as np
from hmmlearn import hmm


In [3]:

np.random.seed(42)

model = hmm.GaussianHMM(n_components=3, covariance_type="full")
model.startprob_ = np.array([0.6, 0.3, 0.1])
model.transmat_ = np.array([[0.7, 0.2, 0.1],
                            [0.3, 0.5, 0.2],
                            [0.3, 0.3, 0.4]])
model.means_ = np.array([[0.0, 0.0], [3.0, -3.0], [5.0, 10.0]])
model.covars_ = np.tile(np.identity(2), (3, 1, 1))
X, Z = model.sample(100)


In [4]:
remodel = hmm.GaussianHMM(n_components=3, covariance_type="full", n_iter=100)
remodel.fit(X)

GaussianHMM(covariance_type='full', n_components=3, n_iter=100)

In [5]:
nWorkers = 5    # Number of Channels
nSamples = 45   # Number of Data
nClasses = 4    # Number of Classes

In [8]:

# Number of possible observations
n_observations = 4

# Number of hidden states
n_states = 4

# Generate random observed data (replace with your actual data)
observations = np.random.randint(0, n_observations, size=1000)

# Create a Categorical Hidden Markov Model
model = hmm.CategoricalHMM(n_components=n_states, n_iter=100)

# Fit the model to the observations using the Baum-Welch algorithm
model.fit(observations.reshape(-1, 1))
obs=observations.reshape(-1, 1)


In [8]:
import numpy as np
from hmmlearn import hmm

# Generate synthetic data
np.random.seed(42)
X = np.concatenate([
    np.random.normal(0, 1, (100, 1)),
    np.random.normal(5, 1, (100, 1)),
    np.random.normal(10, 1, (100, 1))
])

# Initialize the Gaussian HMM
model = hmm.GaussianHMM(n_components=3, covariance_type="diag", n_iter=100, random_state=42)

# Fit the model to the data
model.fit(X)

# Predict the hidden states
hidden_states = model.predict(X)

# Print model parameters
print("Means:", model.means_)
print("Covariances:", model.covars_)
print("Transition Matrix:", model.transmat_)


Means: [[ 7.51580881]
 [ 7.57086256]
 [-0.10376863]]
Covariances: [[[7.53156374]]

 [[7.24896795]]

 [[0.81691181]]]
Transition Matrix: [[1.06136123e-03 9.98938639e-01 4.41129774e-19]
 [9.89463121e-01 1.05368787e-02 6.88686876e-19]
 [8.73411719e-03 1.26624615e-03 9.89999637e-01]]


In [16]:
from pomegranate import *
# state1 = State([DiscreteDistribution({'A': 0.5, 'B': 0.5}),
                # NormalDistribution(50, 10)], name='state1')
# NormalDistribution(50, 10)], name='state1'
d1 = DiscreteDistribution({'A' : 0.65, 'C' : 0.05, 'D' : 0.05, 'E' : 0.05, 'F' : 0.05, 'G' : 0.05, 'H' : 0.05, 'I' : 0.05})
d2 = DiscreteDistribution({'A' : 0.05, 'C' : 0.65, 'D' : 0.05, 'E' : 0.05, 'F' : 0.05, 'G' : 0.05, 'H' : 0.05, 'I' : 0.05})
d3 = DiscreteDistribution({'A' : 0.05, 'C' : 0.05, 'D' : 0.65, 'E' : 0.05, 'F' : 0.05, 'G' : 0.05, 'H' : 0.05, 'I' : 0.05})
d4 = DiscreteDistribution({'A' : 0.05, 'C' : 0.05, 'D' : 0.05, 'E' : 0.65, 'F' : 0.05, 'G' : 0.05, 'H' : 0.05, 'I' : 0.05})
d5 = DiscreteDistribution({'A' : 0.05, 'C' : 0.05, 'D' : 0.05, 'E' : 0.05, 'F' : 0.65, 'G' : 0.05, 'H' : 0.05, 'I' : 0.05})
d6 = DiscreteDistribution({'A' : 0.05, 'C' : 0.05, 'D' : 0.05, 'E' : 0.05, 'F' : 0.05, 'G' : 0.65, 'H' : 0.05, 'I' : 0.05})
d7 = DiscreteDistribution({'A' : 0.05, 'C' : 0.05, 'D' : 0.05, 'E' : 0.05, 'F' : 0.05, 'G' : 0.05, 'H' : 0.65, 'I' : 0.05})
d8 = DiscreteDistribution({'A' : 0.05, 'C' : 0.05, 'D' : 0.05, 'E' : 0.05, 'F' : 0.05, 'G' : 0.05, 'H' : 0.05, 'I' : 0.65})


s1 = State(d1, name="s1")
s2 = State(d2, name="s2")
s3 = State(d3, name="s3")
s4 = State(d4, name="s4")
s5 = State(d5, name="s5")
s6 = State(d6, name="s6")
s7 = State(d7, name="s7")
s8 = State(d8, name="s8")

model = HiddenMarkovModel('example')
model.add_states([s1, s2, s3, s4, s5, s6, s7, s8])
model.add_transition(model.start, s1, 0.90)
model.add_transition(model.start, s2, 0.10)
model.add_transition(s1, s1, 0.80)
model.add_transition(s1, s2, 0.20)
model.add_transition(s2, s2, 0.90)
model.add_transition(s2, s3, 0.10)
model.add_transition(s3, s3, 0.70)
model.add_transition(s3, model.end, 0.30)
model.bake()

print (model.log_probability(list('AAACDDDEEEGGHHHHIIII')))
-4.31828085576
print (", ".join(state.name for i, state in model.viterbi(list('AAACDDDEEEGGHHHHIIII'))[1]))


-52.67051117772867
example-start, s1, s1, s1, s2, s3, s3, s3, s3, s3, s3, s3, s3, s3, s3, s3, s3, s3, s3, s3, s3, example-end
